# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Train an agent

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from unityagents import UnityEnvironment
env = UnityEnvironment(file_name='Reacher.app')

In [ ]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
import torch    
import torch.autograd as autograd         
import torch.nn as nn                     
import torch.nn.functional as F           
import torch.optim as optim               
import torch.distributions

import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer

import streamz
import streamz.dataframe

import pdb, gym

hv.extension('bokeh')

In [ ]:
# Setup training progress output
def init_training_progress():

    max_length = 1000000
    rolling_size = 25

    training_stream = streamz.Stream()
    example = pd.DataFrame({'x': [0]}, index=[0])

    training_sdf = streamz.dataframe.DataFrame(training_stream, example=example)

    training_raw_buffer = Buffer(training_sdf, length=max_length)
    training_smooth_buffer = Buffer(training_sdf.x.rolling(rolling_size).median())

    training_raw_dmap = hv.DynamicMap(hv.Curve, streams=[training_raw_buffer]).relabel('raw')
    training_smooth_dmap = hv.DynamicMap(hv.Curve, streams=[training_smooth_buffer]).relabel('smooth')
    
    return training_stream, training_raw_dmap, training_smooth_dmap

In [ ]:
class PolicyNet(nn.Module):
    
    def __init__(self, state_size, action_size):
        super(PolicyNet, self).__init__()
        
        # Hidden layers
        hidden_size = 32
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
        # Output layer of action means
        self.fc3= nn.Linear(hidden_size, action_size)
        
        # Standard deviations approximated seperately
        self.register_parameter('log_sigma', None)
        self.log_sigma = nn.Parameter(-1*torch.ones(action_size), requires_grad=True)        

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        means = torch.tanh(self.fc3(x))
        sigmas = torch.exp(self.log_sigma).expand(means.shape)
        
        return means, sigmas

In [ ]:
class Rollout():
    
    def __init__(self):
        self.start_episode()
    
    def start_episode(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.log_probs = []
        
    def record_decision(self, state, action, log_prob):
        self.states.append(state)
        self.actions.append(action)
        self.log_probs.append(log_prob)        
    
    def record_outcome(self, reward):
        self.rewards.append(reward)
    
    def flatten_trajectories(self):
        
        # Create tensors from states and actions
        states_tensors = torch.stack(self.states)
        actions_tensors = torch.stack(self.actions)
        
        # Calculate future return (at each step, for each trajectory)
        # This is just the cummulative sum of rewards calculated backwards from the episode end
        future_returns = np.cumsum(self.rewards[::-1], axis=0)[::-1].copy()
        future_returns = torch.tensor(future_returns).float()
        
        # Normalise future return (at each step, for each trajectory)
        mean = future_returns.mean(dim=1).unsqueeze(1)
        sigma = future_returns.std(dim=1).unsqueeze(1)
        normalised_future_returns = (future_returns - mean) / (sigma + 1.0e-10)
        
        # Sum the log probabilities over the possible actions (at each step, for each trajectory)
        # We don't differentiate with respect to these, hence we detach them from the computation graph
        original_policy_log_probs = torch.stack(self.log_probs).sum(-1).detach()
        original_policy_probs = torch.exp(original_policy_log_probs)
        
        # Flatten trajectories
        return (states_tensors.view(-1, states_tensors.shape[-1]), 
                actions_tensors.view(-1, actions_tensors.shape[-1]),
                normalised_future_returns.view(-1),
                original_policy_probs.view(-1))


In [ ]:
class Agent():
    
    def __init__(self, state_size, action_size, lr=1e-3, clipping_epsilon=0.1, ppo_epochs=10, minibatch_size=32):
        self.lr = lr
        self.clipping_epsilon = clipping_epsilon
        self.ppo_epochs = ppo_epochs
        self.minibatch_size = minibatch_size
        
        self.policy = PolicyNet(state_size, action_size)
        self.rollout = Rollout()

    def start_episode(self):
        self.rollout.start_episode()

    def act(self, state):
        
        # Derive action distribution from the policy
        m = self.action_distribution_for_states(state)
        action = m.sample()
        log_prob = m.log_prob(action)
        
        # Record decision and return sampled action
        self.rollout.record_decision(state, action, log_prob)
        return action
    
    def record_outcome(self, reward):
        self.rollout.record_outcome(reward)
        
    def action_distribution_for_states(self, states):
        means, sigmas = self.policy.forward(states)
        return torch.distributions.Normal(means, sigmas)
    
    def average_episode_return(self):
        return np.sum([np.mean(r) for r in self.rollout.rewards])
    
    def get_current_policy_probs(self, states, actions):
        
        # For the given state/action pairs, create a distribution from the policy and get the log probs
        current_policy_log_probs = self.action_distribution_for_states(states).log_prob(actions)

        # Sum log probs over the possible actions
        current_policy_log_probs = current_policy_log_probs.sum(-1)
        
        return torch.exp(current_policy_log_probs)
    
    def learn(self):
        
        (states, actions, normalised_future_returns, original_policy_probs) = \
            self.rollout.flatten_trajectories()
        
        # Run through PPO epochs
        optimiser = optim.Adam(self.policy.parameters(), lr=self.lr, eps=1e-5)
        for ppo_epoch in range(self.ppo_epochs):
            
            # Sample the trajectories randomly in mini-batches
            for indices in random_sample(np.arange(100), self.minibatch_size):
                
                # Sample using sample indices
                states_sample = states[indices]
                actions_sample = actions[indices]
                normalised_future_returns_sample = normalised_future_returns[indices]
                original_policy_probs_sample = original_policy_probs[indices]
            
                # Use the current policy to get the probabilities for the sample states and actions
                # We use these to weight the likehoods, allowing resuse of the rollout
                current_policy_probs_sample = self.get_current_policy_probs(states_sample, actions_sample)

                # Define PPO surrogate and clip
                sampling_ratio = current_policy_probs_sample / original_policy_probs_sample
                clip = torch.clamp(sampling_ratio, 1 - self.clipping_epsilon, 1 + self.clipping_epsilon)
                clipped_surrogate = torch.min(
                    sampling_ratio * normalised_future_returns_sample,
                    clip * normalised_future_returns_sample)

                # Average over trajectories and timesteps
                # Apparently this is preferable to summing over timesteps, since we normalised our rewards
                # (also negative since we want to ascend)
                loss = -torch.mean(clipped_surrogate) 

                # Update model
                optimiser.zero_grad()
                loss.backward()
                optimiser.step()
    


In [ ]:
def play_episode_and_learn(env, agent, time_max=1000):
    
    # Run concurrent episode on all environments
    env_info = env.reset(train_mode=True)[brain_name]
    state = torch.from_numpy(env_info.vector_observations).float()
    agent.start_episode()
    for time in range(time_max):
        
        # Calculate actions for all envs
        action = agent.act(state)
        
        # Run through the envs in parallel
        env_info = env.step(action.numpy())[brain_name]
        next_state = torch.from_numpy(env_info.vector_observations).float()
        reward = env_info.rewards
        done = env_info.local_done
        
        # Record the experience tuple with the agent
        agent.record_outcome(reward)
        
        # Advance
        state = next_state

        # We want rectangular input to network, so if any finish early we finish all early
        if np.any(done):
            print("Someone finished")
            break
    
    # Instruct agent to learn from the episode
    agent.learn()

In [ ]:
# Create policy and agent
state_size = brain.vector_observation_space_size
action_size = brain.vector_action_space_size
agent = Agent(state_size, action_size, lr=3e-4)
episode_index = 0

In [ ]:
# Display training progress
stream, smooth, raw = init_training_progress()
layout = (smooth * raw)
layout.opts(
    opts.Curve(width=900, height=300, show_grid=True, tools=['hover'])
)

In [ ]:
# Train
num_episodes = 10
time_max = 1000

for _ in range(num_episodes):
    
    play_episode_and_learn(env, agent, time_max=time_max)
    
    normalised_return = (agent.average_episode_return() / time_max) * 1000
    stream.emit( pd.DataFrame({'x': normalised_return}, index=[episode_index]) )
    episode_index += 1


In [ ]:
# Show params
for x in agent.subagents[0].policy.parameters():
    print(x)


In [ ]:
# Run agent in the environment
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment
states = torch.from_numpy(env_info.vector_observations).float()
num_agents = len(env_info.agents)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
for i in range(100):
    actions = agent.act(states)
    env_info = env.step(actions.numpy())[brain_name]
    rewards = env_info.rewards                         # get reward (for each agent)
    states = torch.from_numpy(env_info.vector_observations).float()
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    if np.any(dones):                                  # exit loop if episode finished
        break

print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
# Save to disk
for i, subagent in enumerate(agent.subagents):  
    torch.save(subagent.policy, 'subagent.'+str(i)+'.policy')

In [ ]:
# Load from disk
for i, subagent in enumerate(agent.subagents):  
    subagent.policy = torch.load('subagent.'+str(i)+'.policy')
    subagent.policy.eval()

In [ ]:
def random_sample(indices, batch_size):
    indices = np.asarray(np.random.permutation(indices))
    batches = indices[:len(indices) // batch_size * batch_size].reshape(-1, batch_size)
    for batch in batches:
        yield batch
    r = len(indices) % batch_size
    if r:
        yield indices[-r:]

In [ ]:
for minibatch_indices in random_sample(np.arange(100), 32):
    print(minibatch_indices)